In [4]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas

In [70]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [77]:
keyword = 'caffeine'

In [118]:
def ScrapPubMed(keyword:str, pagina:int):
    link = f"https://pubmed.ncbi.nlm.nih.gov/?term={keyword.lower()}&filter=datesearch.y_1&sort=date&size=20&page={pagina}"
    req = requests.get(link)
    soup = BeautifulSoup(req.text, 'html.parser')

    soup = soup.find_all('article', attrs={'class': 'full-docsum'})
    #Pega a data e link da noticia
    soup = [divs.find('div', attrs={'class': 'docsum-wrap'}) for divs in soup]

    noticeName = []
    noticeLink = []
    noticeDataRef = []

    for infos in soup:
        #Pegar o link das noticias

        docontent = infos.find('div', class_={'docsum-content'})
        noticiaCode = docontent.find('a', class_={'docsum-title'})
        Code = noticiaCode['href'] if noticiaCode else None
        noticiaLink = 'https://pubmed.ncbi.nlm.nih.gov'+Code
        noticeLink.append(noticiaLink)

        #Pegar o nome das noticias
        name = noticiaCode.text.strip()
        noticeName.append(name)
        data_ref = docontent.find('div', class_='docsum-citation full-citation').find('span', class_='docsum-journal-citation full-journal-citation').text
        noticeDataRef.append(data_ref) 

    df = {'NoticeName': noticeName, 'noticeLink': noticeLink, 'DataRef':noticeDataRef}
    df = pd.DataFrame(df)

    return df

In [119]:
def lenPages(keyword):
    link = f"https://pubmed.ncbi.nlm.nih.gov/?term={keyword.lower()}&filter=datesearch.y_1&sort=date&size=20"
    req = requests.get(link)
    soup = BeautifulSoup(req.text, 'html.parser')
    total_pages = soup.find_all('label', attrs={'class': 'of-total-pages'})
    string = total_pages[0].text
    match = re.search(r'\d[\d,]*', string)
    if match:
        # Remove as vírgulas e converte para float
        numero = int(match.group().replace(',', ''))
        return numero
    else:
        return None

In [122]:
paginas = lenPages(keyword)

In [123]:
paginas

74

In [124]:
dfs = []

In [125]:
for i in range(paginas):
    df = ScrapPubMed(keyword, pagina=i)
    dfs.append(df)

In [128]:
df_concatenado = pd.concat(dfs, ignore_index=True)

In [159]:
df_concatenado.head(5)

,NoticeName,noticeLink,DataRef
0,Double-Helix Electrode Ion Funnel: A New Ion F...,https://pubmed.ncbi.nlm.nih.gov/39801207/,Anal Chem. 2025 Jan 13. doi: 10.1021/acs.analc...
1,Sugar conditioning combined with nectar nonsug...,https://pubmed.ncbi.nlm.nih.gov/39800782/,Sci Rep. 2025 Jan 12;15(1):1756. doi: 10.1038/...
2,Obesity as an influencing factor for the occur...,https://pubmed.ncbi.nlm.nih.gov/39800623/,Nutr Metab Cardiovasc Dis. 2024 Dec 17:103836....
3,Metabolomic Analysis of Patients With Patent F...,https://pubmed.ncbi.nlm.nih.gov/39799415/,Rapid Commun Mass Spectrom. 2025 Mar 30;39(6):...
4,Advancement in synthetic gene circuits enginee...,https://pubmed.ncbi.nlm.nih.gov/39798857/,Biotechnol Adv. 2025 Jan 9;79:108518. doi: 10....


In [130]:
df_concatenado.to_excel('teste.xlsx')

In [131]:
req = requests.get('https://pubmed.ncbi.nlm.nih.gov/39801207/')

In [133]:
soup = BeautifulSoup(req.text, 'html.parser')

In [134]:
soup

<!DOCTYPE html>

<html lang="en">
<head itemscope="" itemtype="http://schema.org/WebPage" prefix="og: http://ogp.me/ns#">
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!-- Mobile properties -->
<meta content="True" name="HandheldFriendly"/>
<meta content="320" name="MobileOptimized"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://cdn.ncbi.nlm.nih.gov" rel="preconnect"/>
<link href="https://www.ncbi.nlm.nih.gov" rel="preconnect"/>
<link href="https://www.google-analytics.com" rel="preconnect"/>
<link href="https://cdn.ncbi.nlm.nih.gov/pubmed/b4540a7b-d925-4747-955d-e13551e0ee58/CACHE/css/output.5ecf62baa0fa.css" rel="stylesheet" type="text/css"/>
<link href="https://cdn.ncbi.nlm.nih.gov/pubmed/b4540a7b-d925-4747-955d-e13551e0ee58/CACHE/css/output.452c70ce66f7.css" rel="stylesheet" type="text/css"/>
<link href="https://cdn.ncbi.nlm.nih.gov/pubmed/b4540a7b-d925-4747-955d-e13551e0ee58/CACHE/css/output.7bf600913

In [148]:
abstract = soup.find_all('div', attrs={'class': 'abstract-content selected'})

In [160]:
abstract

[<div class="abstract-content selected" id="eng-abstract">
 <p>
       
       The development of an atmospheric pressure interface (API) with a high ion transfer efficiency and wide mass range is advantageous for the performance improvement of mass spectrometry (MS) instruments. In this work, a novel ion guide, namely, the double-helix electrode ion funnel (DHE-IF), has been developed to enhance the ion transmission over a wide mass range in the rough vacuum region. The DHE-IF consists of two funnel-shaped helix electrodes. There are almost no potential "traps" along the central axis of DHE-IF due to the continuous double-helix electrode structure compared to the stacked ring ion funnel. The electrode design of the DHE-IF assembly was guided by ion trajectory simulations. After being fabricated, DHE-IF was integrated into an ESI-TOF-MS platform for tests. A conventional stacked ring ion funnel (IF) was also tested for comparison. The experimental results showed that DHE-IF extended th

In [150]:
resumo = abstract[0].find('p')

In [155]:
resumo_final = resumo.text.strip()

In [156]:
resumo_final

'The development of an atmospheric pressure interface (API) with a high ion transfer efficiency and wide mass range is advantageous for the performance improvement of mass spectrometry (MS) instruments. In this work, a novel ion guide, namely, the double-helix electrode ion funnel (DHE-IF), has been developed to enhance the ion transmission over a wide mass range in the rough vacuum region. The DHE-IF consists of two funnel-shaped helix electrodes. There are almost no potential "traps" along the central axis of DHE-IF due to the continuous double-helix electrode structure compared to the stacked ring ion funnel. The electrode design of the DHE-IF assembly was guided by ion trajectory simulations. After being fabricated, DHE-IF was integrated into an ESI-TOF-MS platform for tests. A conventional stacked ring ion funnel (IF) was also tested for comparison. The experimental results showed that DHE-IF extended the transmission window of the IF and improved the efficiency of the simultaneou

'The development of an atmospheric pressure interface (API) with a high ion transfer efficiency and wide mass range is advantageous for the performance improvement of mass spectrometry (MS) instruments. In this work, a novel ion guide, namely, the double-helix electrode ion funnel (DHE-IF), has been developed to enhance the ion transmission over a wide mass range in the rough vacuum region. The DHE-IF consists of two funnel-shaped helix electrodes. There are almost no potential "traps" along the central axis of DHE-IF due to the continuous double-helix electrode structure compared to the stacked ring ion funnel. The electrode design of the DHE-IF assembly was guided by ion trajectory simulations. After being fabricated, DHE-IF was integrated into an ESI-TOF-MS platform for tests. A conventional stacked ring ion funnel (IF) was also tested for comparison. The experimental results showed that DHE-IF extended the transmission window of the IF and improved the efficiency of the simultaneous transferring of low and medium m/z ions. In addition, the intensities of caffeine ions (m/z = 195) and reserpine ions (m/z = 609) were enhanced by more than 50% and 10%, respectively. These values were compared with the results obtained by the IF. The DHE-IF is expected to be widely used as an ion import device in MS instruments, which is due to its improved performance and advantages in, e.g., integration and power supply design.'

In [161]:
ifa = 'caffeine'

In [162]:
req = requests.get('https://store.usp.org/searchresults?Ntt={ifa}*&Rdm=869&searchType=simple&type=search')

In [163]:
req

<Response [200]>

In [164]:
soup = BeautifulSoup(req.text, 'html.parser')

In [165]:
soup


<!DOCTYPE html>

<html data-bind="setContextVariable:{name:'masterViewModel', value:$data}" dir="ltr" id="oracle-cc" lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta charset="utf-8"/>
<script nonce="" type="text/javascript">
    window.isPreviewMode = false ;
    if (window.isPreviewMode === false ){
      var bActiveX;
      try {
        new ActiveXObject('Microsoft.XMLHTTP');
        bActiveX = true;
      }
      catch(e) { bActiveX = false; }
      if (window.XMLHttpRequest || bActiveX) {
        var xhttp = window.XMLHttpRequest ? new XMLHttpRequest() : new ActiveXObject('Microsoft.XMLHTTP');
        xhttp.onreadystatechange = function() {
          if (this.readyState == 4 && this.status == 200) {
            window.endpointRegistry = xhttp.response ;
          }
        };
        xhttp.open("GET", "/ccstoreui/v1/registry", true);
        xhttp.responseType = 'json';
        xhttp.setRequestHeader("X-CCProfileType", "storefrontUI");
